<a href="https://colab.research.google.com/github/nreichen/eecs448-CollegeSubredditsCommentGeneration/blob/main/subredditValidation/Cornell_test_val.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os 
GOOGLE_DRIVE_PATH_AFTER_MY_DRIVE = "448 Group: Jedi Order/models/Cornell"
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MY_DRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))

Mounted at /content/drive
['checkpoints-Cornell', 'train_post_set.pickle', 'test_post_set.pickle', 'val_post_set.pickle', 'val_com_set.pickle', 'test_com_set.pickle', 'train_com_set.pickle']


In [2]:
print(GOOGLE_DRIVE_PATH)

drive/My Drive/448 Group: Jedi Order/models/Cornell


In [3]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append(GOOGLE_DRIVE_PATH)

In [4]:
print(GOOGLE_DRIVE_PATH)

drive/My Drive/448 Group: Jedi Order/models/Cornell


In [5]:
import locale
print(locale.getpreferredencoding())

UTF-8


In [6]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [7]:
!pip install transformers
!pip install convokit
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 53.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 94.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 23.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.0/168.0 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 23.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for convokit: filename=convokit-2.5.3-py3-none-any.whl size=2

In [8]:
import pickle
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TrainingArguments, Trainer, DataCollatorForLanguageModeling, EarlyStoppingCallback
from datasets import load_dataset
import nltk
# maybe modify the above to a different model? idk up to you all
# also how to use the gpu command? like the cuda stuff.
# 
from convokit import Corpus, download
from torch.utils.data import DataLoader, IterableDataset, Dataset
import torch
import numpy as np

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [9]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


# Load and Prepare Dataset


In [10]:
seed = 448
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

In [11]:
print(GOOGLE_DRIVE_PATH)

drive/My Drive/448 Group: Jedi Order/models/Cornell


In [12]:
val_post_path = os.path.join(GOOGLE_DRIVE_PATH, "val_post_set.pickle")
val_com_path = os.path.join(GOOGLE_DRIVE_PATH, "val_com_set.pickle")

In [13]:
print(val_post_path)

drive/My Drive/448 Group: Jedi Order/models/Cornell/val_post_set.pickle


In [14]:
# posts_datasets_to_use[0] = pickle.load(open(train_post_path, "rb"))
val_posts = pickle.load(open(val_post_path, "rb"))
# posts_datasets_to_use[2] = pickle.load(open(test_post_path, "rb"))
# comments_datasets_to_use[0] = pickle.load(open(train_com_path, "rb"))
val_comments = pickle.load(open(val_com_path, "rb"))
# comments_datasets_to_use[2] = pickle.load(open(test_com_path, "rb"))

In [15]:
for pt in val_posts:
  print(pt)

<bos> Hey everyone! For those of you who took BIOMI 2900, or General Microbiology, did you take the discussion? If so, did it help at all? Trying to decide if I should or not. I'm not a microbiology major but I'm in a microbio lab at the moment so some of it does make sense already. Thanks! <eos> 
<bos> as above <eos> 
<bos> Hello everyone, I'm an accepted sophomore transfer to ILR for this fall. I was a guaranteed transfer so I saved some money and went to community college (but took 18 credits to compensate). Super excited for cornell and would love to hear anyone's advice or thoughts on anything! Any advice/thoughts on:  -being a transfer at cornell -being in ILR at cornell -cornell in general -student life (clubs, parties etc) -housing (i'm going to try to request west campus...hows this for someone whos never lived at college? lame, i know)  -the weather (I'm from south PA)  Anything is appreciated. Thanks! <eos> 
<bos> Hi everyone, inc freshman here. I will be on campus over the 

In [16]:
print(val_posts)

["<bos> Hey everyone! For those of you who took BIOMI 2900, or General Microbiology, did you take the discussion? If so, did it help at all? Trying to decide if I should or not. I'm not a microbiology major but I'm in a microbio lab at the moment so some of it does make sense already. Thanks! <eos> ", '<bos> as above <eos> ', "<bos> Hello everyone, I'm an accepted sophomore transfer to ILR for this fall. I was a guaranteed transfer so I saved some money and went to community college (but took 18 credits to compensate). Super excited for cornell and would love to hear anyone's advice or thoughts on anything! Any advice/thoughts on:  -being a transfer at cornell -being in ILR at cornell -cornell in general -student life (clubs, parties etc) -housing (i'm going to try to request west campus...hows this for someone whos never lived at college? lame, i know)  -the weather (I'm from south PA)  Anything is appreciated. Thanks! <eos> ", "<bos> Hi everyone, inc freshman here. I will be on campu

Load the trained model

In [17]:
model_file_path = "checkpoints-Cornell"

In [18]:
OUTPUT_DIR = os.path.join(GOOGLE_DRIVE_PATH, model_file_path)
model = GPT2LMHeadModel.from_pretrained(OUTPUT_DIR)

In [19]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2", bos_token = "<bos>", eos_token ="<eos>", truncation_side='right')
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


1

^^^^^^run if needed 

In [20]:
#https://colab.research.google.com/github/borisdayma/huggingtweets/blob/master/huggingtweets-demo.ipynb#scrollTo=ZSCf6QyF8AG-
# post_in = "I want to go to hotel management school for rural bed and breakfasts. What experiences or classes should I seek out?"
post_in = "Hey everyone! For those of you who took BIOMI 2900, or General Microbiology, did you take the discussion? If so, did it help at all? Trying to decide if I should or not. I'm not a microbiology major but I'm in a microbio lab at the moment so some of it does make sense already. Thanks!"
post_with_bos = post_in + " TL;DR"
encoded_prompt = tokenizer(post_in, return_tensors="pt").input_ids
encoded_prompt = encoded_prompt.to(model.device)

# prediction
output_sequences = model.generate(
    input_ids=encoded_prompt,
    max_new_tokens=75,
    min_new_tokens=10,
    temperature=1.,
    top_p=0.95,
    do_sample=True,
    num_return_sequences=3
)

generated_sequences = []
print(len(output_sequences))
torch.unique(output_sequences, dim=0)
print(len(output_sequences))

# decode prediction
for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
    generated_sequence = generated_sequence.tolist()
    text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True, skip_special_tokens=True)
    #if not ALLOW_NEW_LINES:
        #limit = text.find('\n')
        #text = text[: limit if limit != -1 else None]
    generated_sequences.append(text.strip())

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


3
3


In [21]:
# print(generated_sequences)
for sent in generated_sequences:
  print(sent.replace(post_in, '').replace('TL;DR', ''))

   The discussion at the end of the class was pretty awesome and definitely helped.  I took it in general and I will definitely recommend it to others if you're interested in the science aspect of CS/MATH 2800 and don't think your stats or your general requirements won't be a big deal. You should learn the basics of CS/
   You could do something similar if you have the right resources to do so. The class itself is quite interesting so you're better off keeping the class as easy as possible. There's a video in a different section but the gist is the textbook is very accessible and is very easy. I didn't have any difficulty with the discussion (I got pretty
   If you had taken the introductory macro biology course, did you take any chemistry class? I took the discussion. I think you've seen the lectures I gave in lecture last semester, so you'll be able to follow them a bit. The chemistry class is very structured and it's very easy to understand most of the chemistry topics I took. But


In [22]:
!pip install rouge-score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24954 sha256=3a0f76b76ccf3245da79989e588ae1d7567c63270be74b23b69970a8a0a476a6
  Stored in directory: /root/.cache/pip/wheels/9b/3d/39/09558097d3119ca0a4d462df68f22c6f3c1b345ac63a09b86e
Successfully built rouge-score


In [23]:
from rouge_score import rouge_scorer
from torch import optim
from torch.utils.data import Dataset
from tqdm import tqdm

In [24]:
# print(post_tensors_data[1])
# print(comments_datasets_to_use[1])
def validation_generate(post):
  # print(post)
  # post_with_bos = "<bos>" + post + " TL;DR"
  post_with_bos = post + " TL;DR"
  encoded_prompt = tokenizer(post_with_bos, return_tensors="pt").input_ids
  encoded_prompt = encoded_prompt.to(model.device)
  # print(encoded_prompt)

  # prediction
  output_sequences = model.generate(
    input_ids=encoded_prompt,
    max_new_tokens=75,
    min_new_tokens=10,
    temperature=1.,
    top_p=0.95,
    do_sample=True,
    num_return_sequences=1
  )
    
  generated_sequences = []
  # decode prediction
  for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
    generated_sequence = generated_sequence.tolist()
    text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True, skip_special_tokens=True)
    generated_sequences.append(text.strip())
  
  # print(generated_sequences)
  return generated_sequences

In [25]:
val_len = len(val_comments)
val_dataset = [{} for sub in range(val_len)]

for cnt, item in enumerate(val_comments):
  val_dataset[cnt]['reference_summary'] = item.replace("<bos>", "").replace("<eos>", "")


print(val_dataset[1]['reference_summary'])

 yes cya in jail loser  


In [26]:
for cnt, post in enumerate(val_posts):
  # print(post)
  generated_sequences = validation_generate(post)

  for sent in generated_sequences:
    sent = sent.replace(post, '').replace('TL;DR', '')
    val_dataset[cnt]['generated_text'] = sent
    # print(sent)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

**Calculating the Rouge score**

In [27]:
!pip3 install Rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [28]:
generated_text = val_dataset[7]['generated_text']
reference_summary = val_dataset[7]['reference_summary']
print("GENERATED COMMENTS: {}\nReference COMMENTS:{}".format(generated_text, reference_summary))

GENERATED COMMENTS: Any experience with this class and its difficulty? Also compared to other Lin Alg classes Professor is Andy Borum if that makes a difference Thanks     I would say you should take it with a professor who is pretty knowledgeable in general relativity. If you are interested in applying a theory in the physics of relativity, it is a great course to take. I also see there are a ton of things you need to know about the theoretical principles of relativity, so I would recommend checking out the course. If you are interested in
Reference COMMENTS: Took it fall ‘15, my freshman year. Much easier than the other Lin alg classes. It’s much more applied and focuses less on proofs and theoretical things, instead mostly is about “can you carry out this process” Of course, I had a different prof so your experience will probably be different.  


In [29]:
from rouge import Rouge

# create a Rouge object
rouge = Rouge()

# initialize scores for each variant of ROUGE
scores_rouge1 = {'recall': 0, 'precision': 0, 'fmeasure': 0}
scores_rouge2 = {'recall': 0, 'precision': 0, 'fmeasure': 0}
scores_rougeL = {'recall': 0, 'precision': 0, 'fmeasure': 0}

for idx in range(val_len):
    generated_text = val_dataset[idx]['generated_text']
    reference_summary = val_dataset[idx]['reference_summary']
    
    scores = rouge.get_scores(generated_text, reference_summary, avg=True)
    
    # compute ROUGE-1 scores
    scores_rouge1['recall'] += scores['rouge-1']['r']
    scores_rouge1['precision'] += scores['rouge-1']['p']
    scores_rouge1['fmeasure'] += scores['rouge-1']['f']
    
    # compute ROUGE-2 scores
    # scores = rouge.get_scores(generated_text, reference_summary, avg=True, rouge_n=(2, 2))
    scores_rouge2['recall'] += scores['rouge-2']['r']
    scores_rouge2['precision'] += scores['rouge-2']['p']
    scores_rouge2['fmeasure'] += scores['rouge-2']['f']
    
    # compute ROUGE-L scores
    # scores = rouge.get_scores(generated_text, reference_summary, avg=True, rouge_l=True)
    scores_rougeL['recall'] += scores['rouge-l']['r']
    scores_rougeL['precision'] += scores['rouge-l']['p']
    scores_rougeL['fmeasure'] += scores['rouge-l']['f']

# compute the average scores over all examples
n_examples = val_len
scores_rouge1 = {k: v/n_examples for k, v in scores_rouge1.items()}
scores_rouge2 = {k: v/n_examples for k, v in scores_rouge2.items()}
scores_rougeL = {k: v/n_examples for k, v in scores_rougeL.items()}

# print the overall scores
print("Overall ROUGE-1 scores:", scores_rouge1)
print("Overall ROUGE-2 scores:", scores_rouge2)
print("Overall ROUGE-L scores:", scores_rougeL)

Overall ROUGE-1 scores: {'recall': 0.23143145634633647, 'precision': 0.19651358519692935, 'fmeasure': 0.1976551238529869}
Overall ROUGE-2 scores: {'recall': 0.029705042442088308, 'precision': 0.02429397497902146, 'fmeasure': 0.024176494032816537}
Overall ROUGE-L scores: {'recall': 0.2014132595751676, 'precision': 0.17140004207853526, 'fmeasure': 0.17201980875771386}


**Calculating the sentiment analysis accuracy**

In [30]:
!pip3 install vaderSentiment

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 4.0 MB/s eta 0:00:00


In [31]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [32]:
def find_label(scores):
  if scores['pos'] > scores['neg'] and scores['pos'] > scores['neu']:
    dominant_label = 'positive'
  elif scores['neg'] > scores['pos'] and scores['neg'] > scores['neu']:
    dominant_label = 'negative'
  else:
    dominant_label = 'neutral'
  
  return dominant_label

In [33]:
acc = 0
sid = SentimentIntensityAnalyzer()
for idx in range(val_len):
  # print(val_dataset[idx]['reference_summary'])
  # print(type(val_dataset[idx]['reference_summary']))
  sent_gt = sid.polarity_scores(val_dataset[idx]['reference_summary'])
  sent_generated_comment = sid.polarity_scores(val_dataset[idx]['generated_text'])

  gt_label = find_label(sent_gt)
  generated_comment_label = find_label(sent_generated_comment)

  if gt_label == generated_comment_label:
    acc += 1

  # print("Reference comments: {}".format(val_dataset[idx]['reference_summary']))
  # print(sent_gt)
  # print(gt_label)
  # print("Generated comments: {}".format(val_dataset[idx]['generated_text']))
  # print(sent_generated_comment)
  # print(generated_comment_label)
  # if idx == 15:
  #   break

print("The number of generated comments that have aligned sentiments: {}".format(acc))
print("The number of posts in the validation set: {}".format(val_len))
print("The accuracy of sentiments of the generated comments: {}".format(acc / val_len))

The number of generated comments that have aligned sentiments: 812
The number of posts in the validation set: 827
The accuracy of sentiments of the generated comments: 0.9818621523579202


**Readability validation test**

Automated readability index

In [34]:
!pip3 install textstat

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 37.9 MB/s eta 0:00:00


Flesch-Kincaid Grade Level: The Flesch-Kincaid Grade Level measures the approximate grade level of a text, based on its sentence length and average number of syllables per word.

In [35]:
import textstat

In [36]:
grade_level = 0.0
for idx in range(val_len):
  sc = textstat.flesch_kincaid_grade(val_dataset[idx]['generated_text'])
  # print(sc)
  grade_level += sc

grade_level /= val_len
print("Flesch-Kincaid Grade Level:", grade_level)

Flesch-Kincaid Grade Level: 6.474486094316815


Gunning Fog Index: The Gunning Fog Index measures the approximate years of education needed to understand a text, based on its sentence length and percentage of complex words.

In [37]:
fog_index = 0.0
for idx in range(val_len):
  sc = textstat.gunning_fog(val_dataset[idx]['generated_text'])
  # print(sc)
  fog_index += sc

fog_index /= val_len
print("Gunning Fog Index:", fog_index)

Gunning Fog Index: 8.60303506650544


Automated Readability Index (ARI): The Automated Readability Index measures the approximate grade level of a text, based on its average number of characters per word and average number of words per sentence.



In [38]:
ari = 0.0
for idx in range(val_len):
  sc = textstat.automated_readability_index(val_dataset[idx]['generated_text'])
  # print(sc)
  ari += sc

ari /= val_len
print("Automated Readability Index:", ari)

Automated Readability Index: 7.110278113663842
